In [1]:
import time
import concurrent.futures
import requests
import pandas as pd

In [2]:
start_time = time.time()
print(f"start time: {time.strftime('%x %X')}")

start time: 09/07/20 21:36:18


In [3]:
size_chart_exists = '''id="size-chart-url" data-header="Size Chart" data-sizeChartActionType="a-modal" class="a-size-small a-link-normal"'''
def size_chart(asin):
    f = requests.get("https://www.amazon.com/dp/" + asin)
    if f.status_code == 200:
        if int(str(f.content).find(size_chart_exists)) == -1 :
            return "Size chart widget does not exist"
        else:
            return "Size chart widget exist"
    else:
        return "Page Error"

In [4]:
asins = pd.read_csv(r"C:\Users\akmarmu\Documents\python_projects\fit_analysis\asins.csv").iloc[:,0].to_list()
len(asins)

1816

In [5]:
# # MT method
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     results = executor.map(size_chart, asins)
# result_df = pd.DataFrame()
# result_df['asin'] = asins
# result_df['size chart widget status'] = list(results)
# result_df.to_csv("asins_size_widget_status_mt.csv", index=False)
# end_time = time.time()
# minutes = (end_time - start_time) // 60
# seconds = (end_time - start_time) % 60
# print(f"end time: {time.strftime('%x %X')}")
# print(f"Approx. time taken : {round(minutes)} minutes {round(seconds)} seconds")

In [6]:
result = []
for i in range(len(asins)):
    result.append(size_chart(asins[i]))

In [9]:
# Normal method
result_df = pd.DataFrame()
result_df['asin'] = asins
result_df['size chart widget status'] = result
result_df.to_csv("asins_size_widget_status.csv", index=False)
end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f"end time: {time.strftime('%x %X')}")
print(f"Approx. time taken : {round(minutes)} minutes {round(seconds)} seconds")

end time: 09/07/20 23:12:43
Approx. time taken : 96 minutes 25 seconds


In [10]:
# Importing packages
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import time
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
)


print("\n\n")


def chrome():
    """
    This function returns a chrome instance
    """
    option = webdriver.ChromeOptions()
    option.add_argument("--ignore-certificate-errors")
    option.add_argument("--ignore-ssl-errors")
    option.add_argument("--safebrowsing-disable-download-protection")
    return webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=option)


def fit(asin):

    try:
        driver.get("https://www.amazon.com/dp/" + asin)
        time.sleep(5)
        driver.find_element_by_xpath("//*[@id = 'fitRecommendationsSection']").click()
        grabbed_element = driver.find_element_by_xpath(
            "//*[@id = 'fitRecommendationsSupportingStatement']/following::*[@id = 'fitRecommendationHistogramTable']"
        ).text
        rt = re.findall(r"\n([,0-9]+)", grabbed_element)
        return rt

    except NoSuchElementException:
        try:
            driver.get("https://www.amazon.com/dp/" + asin)
            time.sleep(5)
            driver.find_element_by_xpath("//*[@id = 'pers_fit_link']").click()
            driver.find_element_by_xpath("//*[contains(text(),'Fit reviews')]").click()
            rt = re.findall(
                r"\n([,0-9]+)",
                driver.find_element_by_xpath(
                    "//*[@id = 'fitRecommendationHistogramTable']"
                ).text,
            )
            return rt
        except NoSuchElementException:
            try:
                driver.get("https://www.amazon.com/dp/" + asin)
                time.sleep(5)
                driver.find_element_by_xpath("//*[@id = 'HIF_link']").click()
                driver.find_element_by_xpath(
                    "//*[contains(text(),'Fit reviews')]"
                ).click()
                rt = re.findall(
                    r"\n([,0-9]+)",
                    driver.find_element_by_xpath(
                        "//*[@id = 'fitRecommendationHistogramTable']"
                    ).text,
                )
                return rt
            except NoSuchElementException:
                return ["Error", "Error", "Error", "Error", "Error"]
            except ElementClickInterceptedException:
                return ["click", "click", "click", "click", "click"]
            except:
                return ["others", "others", "others", "others", "others"]
        except ElementClickInterceptedException:
            return ["click", "click", "click", "click", "click"]
        except:
            return ["others", "others", "others", "others", "others"]

    except ElementClickInterceptedException:
        return ["click", "click", "click", "click", "click"]
    except:
        return ["others", "others", "others", "others", "others"]


start_time = time.time()
print(f"start time: {time.strftime('%x %X')}")
print("")

print("Reading the file, may take few minutes...........")
ds = pd.read_csv(r"C:\Users\akmarmu\Documents\python_projects\fit_analysis\asins.csv")
ds = ds.iloc[:, 0]
ds = ds.drop_duplicates()
print("\n", "Total unique ASINs to check: ", len(ds), "\n", end="\n \n")
driver = chrome()

hi = ds.apply(fit)

driver.quit()
ds = pd.concat([ds, hi], axis=1)
ds.to_csv("result.csv", index=True)

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f"end time: {time.strftime('%x %X')}")
print(
    f"Approx. time taken to get all the values: {round(minutes)} minutes {round(seconds)} seconds"
)





start time: 09/07/20 23:18:59

Reading the file, may take few minutes...........

 Total unique ASINs to check:  951 

 


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - There is no [win32] chromedriver for browser 85.0.4183 in cache
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\akmarmu\.wdm\drivers\chromedriver\win32\85.0.4183.87]


end time: 09/08/20 04:29:43
Approx. time taken to get all the values: 310 minutes 43 seconds
